Exploring how to re-model the safety related data and criticality scores into instances of traditional algorithmic problems and using the solvers provided by Google's OR Tools to handle the same.

In [ ]:
!pip install -U ortools

In [ ]:
# [START program]
"""Modified from Taha 'Introduction to Operations Research', example 6.4-2."""
# [START import]
import numpy as np

from ortools.graph.python import max_flow
# [END import]

def main():
    """MaxFlow example with localities in Chennai and safety scores."""
    # [START solver]
    # Instantiate a SimpleMaxFlow solver.
    locality_network = max_flow.SimpleMaxFlow()
    # [END solver]

    # [START data]
    # Define three parallel arrays: start_localities, end_localities, and the safety scores
    start_localities = np.array([0, 0, 0, 1, 1, 2, 2, 3, 3])
    end_localities = np.array([1, 2, 3, 2, 4, 3, 4, 2, 4])
    safety_scores = np.array([9, 7, 8, 5, 6, 8, 9, 6, 7])  # Higher scores indicate safer localities
    locality_names = ["Adyar", "Mylapore", "Velachery", "Guindy", "T. Nagar"]  # Replace with actual locality names
    # [END data]

    # [START constraints]
    # Add arcs with safety scores as capacities.
    all_arcs = locality_network.add_arcs_with_capacity(start_localities, end_localities, safety_scores)
    # [END constraints]

    # [START solve]
    # Find the maximum safety flow between locality 0 and locality 4.
    status = locality_network.solve(0, 4)
    # [END solve]

    # [START print_solution]
    if status != locality_network.OPTIMAL:
        print("There was an issue with the max safety flow input.")
        print(f"Status: {status}")
        exit(1)

    print("Max safety flow:")
    print("")

    print(" Locality     Safety Flow   Safety Score")
    print("---------------------------------------")
    solution_flows = locality_network.flows(all_arcs)
    for arc, flow, score in zip(all_arcs, solution_flows, safety_scores):
        start_locality = locality_names[locality_network.tail(arc)]
        end_locality = locality_names[locality_network.head(arc)]
        print(f"{start_locality:10}   ->   {end_locality:10}       {flow:3}            {score:3}")
    print("\nSource side min-cut:", locality_network.get_source_side_min_cut())
    print("Sink side min-cut:", locality_network.get_sink_side_min_cut())
    # [END print_solution]

if __name__ == "__main__":
    main()
# [END program]


Max safety flow:

 Locality     Safety Flow   Safety Score
---------------------------------------
Adyar        ->   Mylapore           9              9
Adyar        ->   Velachery          5              7
Adyar        ->   Guindy             8              8
Mylapore     ->   Velachery          3              5
Mylapore     ->   T. Nagar           6              6
Velachery    ->   Guindy             0              8
Velachery    ->   T. Nagar           9              9
Guindy       ->   Velachery          1              6
Guindy       ->   T. Nagar           7              7

Source side min-cut: [0, 2, 1, 3]
Sink side min-cut: [4]


In [ ]:
# [START program]
"""Modification of Simple Travelling Salesman Problem.
"""

# [START import]
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]


# [START data_model]
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Locations and names
    localities = [
        (4, 4, "Chennai Central"),
        (2, 0, "Marina Beach"),
        (8, 0, "Egmore"),
        (0, 1, "Mylapore"),
        (1, 1, "T. Nagar"),
        (5, 2, "Adyar"),
        (7, 2, "Guindy"),
        (3, 3, "Besant Nagar"),
        (6, 3, "Velachery"),
        (5, 5, "Kotturpuram"),
        (8, 5, "Anna Nagar"),
        (1, 6, "Nungambakkam"),
        (2, 6, "Kodambakkam"),
        (3, 7, "Royapettah"),
        (6, 7, "Thiruvanmiyur"),
        (0, 8, "Triplicane"),
        (7, 8, "Aminjikarai")
    ]
    # Convert locations in meters using a city block dimension of 114m x 80m.
    data["locations"] = [(l[0] * 114, l[1] * 80, l[2]) for l in localities]
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data
    # [END data_model]


# [START distance_callback]
def create_distance_callback(data, manager):
    """Creates callback to return distance between points."""
    distances_ = {}
    index_manager_ = manager
    # precompute distance between location to have distance callback in O(1)
    for from_counter, from_node in enumerate(data["locations"]):
        distances_[from_counter] = {}
        for to_counter, to_node in enumerate(data["locations"]):
            if from_counter == to_counter:
                distances_[from_counter][to_counter] = 0
            else:
                distances_[from_counter][to_counter] = abs(
                    from_node[0] - to_node[0]
                ) + abs(from_node[1] - to_node[1])

    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = index_manager_.IndexToNode(from_index)
        to_node = index_manager_.IndexToNode(to_index)
        return distances_[from_node][to_node]

    return distance_callback
    # [END distance_callback]


# [START solution_printer]
def print_solution(manager, routing, assignment, data):
    """Prints assignment on console with locality names."""
    print(f"Objective: {assignment.ObjectiveValue()}")
    index = routing.Start(0)
    plan_output = "Route for user 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        locality_name = data["locations"][manager.IndexToNode(index)][2]
        plan_output += f" {locality_name} ->"
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    last_locality_name = data["locations"][manager.IndexToNode(index)][2]
    plan_output += f" {last_locality_name}\n"
    plan_output += f"Distance of the route: {route_distance}m\n"
    print(plan_output)
# [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model()
    print(f"Data Model: {data}")
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(
        len(data["locations"]), data["num_vehicles"], data["depot"]
    )
    print(f"Routing Index Manager: {manager}")
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)
    print(f"Routing Model: {routing}")
    # [END routing_model]

    # Create and register a transit callback.
    # [START transit_callback]
    distance_callback = create_distance_callback(data, manager)
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    print(f"Transit Callback Index: {transit_callback_index}")
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    print("Arc Costs Set.")
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    print(f"Search Parameters: {search_parameters}")
    # [END parameters]

    # Solve the problem.
    # [START solve]
    assignment = routing.SolveWithParameters(search_parameters)
    print(f"Solving... Assignment: {assignment}")
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if assignment:
        print_solution(manager, routing, assignment, data)
    # [END print_solution]


if __name__ == "__main__":
    main()
# [END program]


Data Model: {'locations': [(456, 320, 'Chennai Central'), (228, 0, 'Marina Beach'), (912, 0, 'Egmore'), (0, 80, 'Mylapore'), (114, 80, 'T. Nagar'), (570, 160, 'Adyar'), (798, 160, 'Guindy'), (342, 240, 'Besant Nagar'), (684, 240, 'Velachery'), (570, 400, 'Kotturpuram'), (912, 400, 'Anna Nagar'), (114, 480, 'Nungambakkam'), (228, 480, 'Kodambakkam'), (342, 560, 'Royapettah'), (684, 560, 'Thiruvanmiyur'), (0, 640, 'Triplicane'), (798, 640, 'Aminjikarai')], 'num_vehicles': 1, 'depot': 0}
Routing Index Manager: <ortools.constraint_solver.pywrapcp.RoutingIndexManager; proxy of <Swig Object of type 'operations_research::RoutingIndexManager *' at 0x7b43b90d5a70> >
Routing Model: <ortools.constraint_solver.pywrapcp.RoutingModel; proxy of <Swig Object of type 'operations_research::RoutingModel *' at 0x7b43b90d7db0> >
Transit Callback Index: 1
Arc Costs Set.
Search Parameters: first_solution_strategy: PATH_CHEAPEST_ARC
savings_neighbors_ratio: 1
savings_max_memory_usage_bytes: 6000000000
savings